In [5]:
import PyPDF2 as pdf2
import joblib
import re


import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import  word_tokenize
from nltk.probability import FreqDist
from nltk import  blankline_tokenize
from nltk.util import bigrams, trigrams,ngrams
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller 
from spacy.matcher import Matcher 

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

import spacy
nlp=spacy.load('en_core_web_sm')

In [1]:
from tabula.io import read_pdf
bok = "docs/PMBOK 5th.pdf"
pmi = "docs/PMIPracticeStandardforProjectRiskManagement.pdf"
# [top,left,bottom,width]
box_bok = [1.425, 0, 25.386, 19.947]
box_pmi = [3.366, 0, 27.25, 24]
fc=28.28
for i in range(0, len(box_bok)):
    box_bok[i] *= fc
    
for i in range(0, len(box_pmi)):
    box_pmi[i] *= fc

t_bok = read_pdf(bok, pages=list(range(335, 380)),area=[box_bok],output_format="json", stream=True)
t_pmi = read_pdf(pmi, pages=list(range(13, 67)),area=[box_pmi],output_format="json", stream=True)

In [3]:
[box_bok]

[[40.299, 0.0, 717.91608, 564.10116]]

# Get Data Json Format

In [23]:
all_data_bok=[]
for i in range(len(t_bok)):
    all_data_bok.append(t_bok[i]['data'])
    
    
all_data_pmi=[]
for i in range(len(t_pmi)):
    all_data_pmi.append(t_pmi[i]['data'])

# Flatten the list

In [46]:
data_bok = [item for sublist in all_data_bok for item in sublist]

data_pmi = [item for sublist in all_data_pmi for item in sublist]

In [70]:
text_bok=[]
for i in range(len(data_bok)):
    text_bok.append(data_bok[i][0]['text'])
    
    
text_pmi=[]
for i in range(len(data_pmi)):
    text_pmi.append(data_pmi[i][0]['text'])

In [71]:
text_bok = [i for i in text_bok if i !='']
text_pmi = [i for i in text_pmi if i !='']

In [72]:
text_bok_lda = [i for i in text_bok if i !='']
text_pmi_lda = [i for i in text_pmi if i !='']

# String Text

In [73]:
text_bok = " ".join([str(elem) for elem in text_bok])
text_pmi = " ".join([str(elem) for elem in text_pmi])

## Remove newlines & Tabs

In [74]:
def remove_newlines_tabs(text):    
    # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
    Formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ')
    return Formatted_text
text_bok = remove_newlines_tabs(text_bok)
text_pmi = remove_newlines_tabs(text_pmi)

## Case Conversion

In [75]:
def lower_casing_text(text):
    text = text.lower()
    return text
text_bok = lower_casing_text(text_bok)
text_pmi = lower_casing_text(text_pmi)

##  Remove Special Characters

In [76]:
def removing_special_characters(text):
    text="".join([i for i in text if i.isalpha() or i==" " or i =='.'])
    return text
text_bok = removing_special_characters(text_bok)
text_pmi = removing_special_characters(text_pmi)

## Sentence Tokenization

In [78]:
def sentences(text):
    # split sentences and questions
    text = re.split('[.]', text)
    clean_sent = []
    for sent in text:
        clean_sent.append(sent)
    return clean_sent
sent_bok = sentences(text_bok)
sent_pmi = sentences(text_pmi)
joblib.dump(sent_bok,'bok')
joblib.dump(sent_pmi,'pmi')

['pmi']

##  Tokenization

In [55]:
def tokenization(text):
    tokens=word_tokenize(text)
    return tokens
text_bok = tokenization(text_bok)
text_pmi = tokenization(text_pmi)

##  Bigram

In [56]:
bi_bok = list(nltk.bigrams(text_bok))

bi_pmi = list(nltk.bigrams(text_pmi))

In [57]:
bi_bok

[('project', 'risk'),
 ('risk', 'management'),
 ('management', 'project'),
 ('project', 'risk'),
 ('risk', 'management'),
 ('management', 'includes'),
 ('includes', 'the'),
 ('the', 'processes'),
 ('processes', 'of'),
 ('of', 'conducting'),
 ('conducting', 'risk'),
 ('risk', 'management'),
 ('management', 'planning'),
 ('planning', 'identification'),
 ('identification', 'analysis'),
 ('analysis', 'response'),
 ('response', 'planning'),
 ('planning', 'and'),
 ('and', 'controlling'),
 ('controlling', 'risk'),
 ('risk', 'on'),
 ('on', 'a'),
 ('a', 'project'),
 ('project', 'the'),
 ('the', 'objectives'),
 ('objectives', 'of'),
 ('of', 'project'),
 ('project', 'risk'),
 ('risk', 'management'),
 ('management', 'are'),
 ('are', 'to'),
 ('to', 'increase'),
 ('increase', 'the'),
 ('the', 'likelihood'),
 ('likelihood', 'and'),
 ('and', 'impact'),
 ('impact', 'of'),
 ('of', 'positive'),
 ('positive', 'events'),
 ('events', 'and'),
 ('and', 'decrease'),
 ('decrease', 'the'),
 ('the', 'likelihood')

##  Lemmatization

In [58]:
def lemma(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(doc)
    return texts_out

text_bok_lemma = lemma(bi_bok)
text_pmi_lemma = lemma(bi_pmi)

In [60]:
flat_text_bok_lemma=[]
for i in range(len(bi_bok)):
    flat_text_bok_lemma.append(' '.join(bi_bok[i]))
    
flat_text_pmi_lemma=[]
for i in range(len(bi_pmi)):
    flat_text_pmi_lemma.append(' '.join(bi_pmi[i]))

In [61]:
flat_text_bok_lemma

['project risk',
 'risk management',
 'management project',
 'project risk',
 'risk management',
 'management includes',
 'includes the',
 'the processes',
 'processes of',
 'of conducting',
 'conducting risk',
 'risk management',
 'management planning',
 'planning identification',
 'identification analysis',
 'analysis response',
 'response planning',
 'planning and',
 'and controlling',
 'controlling risk',
 'risk on',
 'on a',
 'a project',
 'project the',
 'the objectives',
 'objectives of',
 'of project',
 'project risk',
 'risk management',
 'management are',
 'are to',
 'to increase',
 'increase the',
 'the likelihood',
 'likelihood and',
 'and impact',
 'impact of',
 'of positive',
 'positive events',
 'events and',
 'and decrease',
 'decrease the',
 'the likelihood',
 'likelihood and',
 'and impact',
 'impact of',
 'of negative',
 'negative events',
 'events in',
 'in the',
 'the project',
 'project figure',
 'figure provides',
 'provides an',
 'an overview',
 'overview of',
 

In [42]:
flat_text_bok_lemma

['project risk',
 'risk management',
 'management project',
 'project risk',
 'risk management',
 'management include',
 'include',
 'process',
 'process',
 'conduct',
 'conduct risk',
 'risk management',
 'management planning',
 'planning identification',
 'identification analysis',
 'analysis response',
 'response planning',
 'planning',
 'control',
 'control risk',
 'risk',
 '',
 'project',
 'project',
 'objective',
 'objective',
 'project',
 'project risk',
 'risk management',
 'management',
 '',
 'increase',
 'increase',
 'likelihood',
 '',
 'impact',
 'impact',
 'positive',
 'positive event',
 'event',
 '',
 'decrease',
 'likelihood',
 '',
 'impact',
 'impact',
 '',
 'negative event',
 'event',
 '',
 'project',
 'project figure',
 'figure provide',
 'provide',
 'overview',
 'overview',
 '',
 'project',
 'project risk',
 'risk management',
 'management process',
 'process',
 '',
 '',
 'follow',
 'follow plan',
 'plan risk',
 'risk',
 'process',
 'process',
 'define',
 'define how'

### Remove doubles

In [43]:
flat_text_bok_lemma = list(dict.fromkeys(flat_text_bok_lemma))
flat_text_pmi_lemma = list(dict.fromkeys(flat_text_pmi_lemma))

### Remove None values 

In [44]:
flat_text_bok_lemma = [i for i in flat_text_bok_lemma if i !='']
flat_text_pmi_lemma = [i for i in flat_text_pmi_lemma if i !='']

In [45]:
flat_text_pmi_lemma

['chapter introduction',
 'introduction project',
 'project management',
 'practice',
 'practice standard',
 'standard',
 'guide',
 'use',
 'tool',
 'tool technique',
 'technique',
 'process',
 'process identifi',
 'identifi',
 'project',
 'management body',
 'body',
 'knowledge',
 'guide fourth',
 'edition',
 'other',
 'standard practice',
 'target',
 'audience',
 'participate',
 'management',
 'include',
 'project manager',
 'manager',
 'project personnel',
 'personnel contract',
 'contract personnel',
 'personnel supervisor',
 'supervisor',
 'other project',
 'project stakeholder',
 'stakeholder',
 'pmi',
 'describe process',
 'process activity',
 'activity input',
 'input',
 'output',
 'specifi',
 'knowledge area',
 'area',
 'provide',
 'provide information',
 'information',
 'signifi',
 'signifi can',
 'can process',
 'process tool',
 'why',
 'can when',
 'when',
 'should',
 'perform',
 'execute',
 'necessary',
 'further',
 'cation',
 'should perform',
 'prescribe',
 'prescribe ho

## TF-IDF

In [24]:
def TF(text):    
    doc = CountVectorizer()
    word_count=doc.fit_transform(text)
    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count)
    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=doc.get_feature_names(),columns=["idf_weights"])
    df_idf.sort_values(by=['idf_weights'],ascending=False).head()
    return df_idf
TF_IDF_bok = TF(flat_text_bok_lemma)
TF_IDF_pmi = TF(flat_text_pmi_lemma)

In [25]:
TF_IDF_bok

,idf_weights
ability,7.433583
able,7.433583
academic,7.944409
accept,6.963579
acceptable,7.656727
...,...
worked,8.349874
workshop,7.944409
would,7.944409
write,7.944409


In [26]:
TF_IDF_pmi

,idf_weights
ability,7.460575
able,7.460575
accept,7.008590
acceptable,6.841536
acceptance,8.020191
...,...
work,7.103900
workassignment,8.307873
workforce,8.307873
workshop,8.713338


## Postagging

In [27]:
pos_bok, pos_pmi = [], []
for token in flat_text_bok_lemma:
    pos_bok.append(nltk.pos_tag([token]))
    
for token in flat_text_pmi_lemma:
    pos_pmi.append(nltk.pos_tag([token]))

In [28]:
pos_bok = [item for sublist in pos_bok for item in sublist]
pos_pmi = [item for sublist in pos_pmi for item in sublist]

In [29]:
pattern = 'NP: {<DT>?<JJ>*<NN>}'
cp = nltk.RegexpParser(pattern)
cs_bok = cp.parse(pos_bok)

cs_pmi = cp.parse(pos_pmi)

In [30]:
cs_pmi

KeyboardInterrupt: 

In [31]:
data = [text_bok_cos, text_pmi_cos]

NameError: name 'text_bok_cos' is not defined

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
vector_matrix = count_vectorizer.fit_transform(data)


tokens = count_vectorizer.get_feature_names()

In [ ]:
import pandas as pd

def create_dataframe(matrix, tokens):

    doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)
x = create_dataframe(vector_matrix.toarray(),tokens)
x

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['doc_1','doc_2'])

# Spacy Rule 

## Rule1

In [32]:
def rule1(text): 
    doc = nlp(text) 
    sent = []
    for token in doc: 
        if (token.pos_=='VERB'):
            phrase = "" 
            for sub_tok in token.lefts: 
                if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON'] ):
                    phrase += sub_tok.text 
                    phrase += " "+token.text
                    for sub_tok in token.rights:
                        if (sub_tok.dep_ in ['dobj']) and (sub_tok.pos_ in ['NOUN','PROPN'] ):
                            phrase += " "+sub_tok.text
                            sent.append(phrase)
    return sent


In [34]:
output_bok_rule1, output_pmi_rule1 = [],[]

for i in sent_bok:
    output_bok_rule1.append(rule1(i))
    

for i in sent_pmi:
    output_pmi_rule1.append(rule1(i))
    
df_rule1_bok = pd.DataFrame({'sentences':sent_bok,'output':output_bok_rule1})
df_rule1_pmi = pd.DataFrame({'sentences':sent_pmi,'output':output_pmi_rule1})

### Separate subject, verb and object for PMBOK

In [35]:
verb_dict = dict()
dis_dict = dict()
dis_list = []

# iterating over all the sentences
for i in range(len(df_rule1_pmi)):
    
    # sentence containing the output
    sentence = df_rule1_pmi.loc[i,'sentences']
    # output of the sentence
    output = df_rule1_pmi.loc[i,'output']
    
    # iterating over all the outputs from the sentence
    for sent in output:
        
        # separate subject, verb and object
        n1 = sent.split()[:1]
        v = sent.split()[1]
        n2 = sent.split()[2:]
        
        # append to list, along with the sentence
        dis_dict = {'sentence':sentence,'Noun1':n1,'Verb':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep_bok_rule1 = pd.DataFrame(dis_list)

In [41]:
df_sep_bok_rule1

,sentence,Noun1,Verb,Noun2
0,chapter introduction project management instit...,[process],identifi,[guide]
1,practice standards targeted audiences particip...,[audiences],participate,[projects]
2,"pmi practice standard describes processes , ac...",[processes],specifi,[area]
3,"pmi practice standard describes processes , ac...",[processes],specifi,"[areaoutputs, specifi, area]"
4,purpose practice standard project risk managem...,[management],provide,[practitioners]
...,...,...,...,...
477,projects seniorlevel sponsor require regular r...,[sponsor],require,[reports]
478,sponsor feedback also motivates project team d...,[feedback],motivates,[team]
479,tools techniques monitor control risks process...,[techniques],monitor,[capabilities]
480,tools techniques monitor control risks process...,[monitoring],requires,[tools]


### Separate subject, verb and object for PMI

In [36]:
verb_dict = dict()
dis_dict = dict()
dis_list = []

# iterating over all the sentences
for i in range(len(df_rule1_bok)):
    
    # sentence containing the output
    sentence = df_rule1_bok.loc[i,'sentences']
    # output of the sentence
    output = df_rule1_bok.loc[i,'output']
    
    # iterating over all the outputs from the sentence
    for sent in output:
        
        # separate subject, verb and object
        n1 = sent.split()[:1]
        v = sent.split()[1]
        n2 = sent.split()[2:]
        
        # append to list, along with the sentence
        dis_dict = {'sentence':sentence,'Noun1':n1,'Verb':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep_pmi_rule1 = pd.DataFrame(dis_list)

## Rule 2 

In [39]:
# function for rule 2
def rule2(text):
    
    doc = nlp(text)

    pat = []
    
    # iterate over tokens
    for token in doc:
        phrase = ''
        # if the word is a subject noun or an object noun
        if (token.pos_ == 'NOUN')\
            and (token.dep_ in ['dobj','pobj','nsubj','nsubjpass']):
            
            # iterate over the children nodes
            for subtoken in token.children:
                # if word is an adjective or has a compound dependency
                if (subtoken.pos_ == 'ADJ') or (subtoken.dep_ == 'compound'):
                    phrase += subtoken.text + ' '
                    
            if len(phrase)!=0:
                phrase += token.text
             
        if  len(phrase)!=0:
            pat.append(phrase)
        
    
    return pat

In [40]:
output_bok_rule2, output_pmi_rule2 = [],[]

for i in sent_bok:
    output_bok_rule2.append(rule2(i))
    

for i in sent_pmi:
    output_pmi_rule2.append(rule2(i))
    
df_rule2_bok = pd.DataFrame({'sentences':sent_bok,'output':output_bok_rule2})
df_rule2_pmi = pd.DataFrame({'sentences':sent_pmi,'output':output_pmi_rule2})

### Separate subject, verb and object for PMBOK

In [185]:
verb_dict = dict()
dis_dict = dict()
dis_list = []

# iterating over all the sentences
for i in range(len(df_rule2_bok)):
    
    # sentence containing the output
    sentence = df_rule2_bok.loc[i,'sentences']
    # output of the sentence
    output = df_rule2_bok.loc[i,'output']
    
    # iterating over all the outputs from the sentence
    for sent in output:
        
        # separate subject, verb and object
        n1 = sent.split()[:1]
        v = sent.split()[1]
        n2 = sent.split()[2:]
        
        # append to list, along with the sentence
        dis_dict = {'sentence':sentence,'Noun1':n1,'Verb':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep_bok_rule2 = pd.DataFrame(dis_list)

### Separate subject, verb and object for PMI

In [187]:
verb_dict = dict()
dis_dict = dict()
dis_list = []

# iterating over all the sentences
for i in range(len(df_rule2_pmi)):
    
    # sentence containing the output
    sentence = df_rule2_pmi.loc[i,'sentences']
    # output of the sentence
    output = df_rule2_pmi.loc[i,'output']
    
    # iterating over all the outputs from the sentence
    for sent in output:
        
        # separate subject, verb and object
        n1 = sent.split()[:1]
        v = sent.split()[1]
        n2 = sent.split()[2:]
        
        # append to list, along with the sentence
        dis_dict = {'sentence':sentence,'Noun1':n1,'Verb':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep_pmi_rule2 = pd.DataFrame(dis_list)

In [189]:
df_sep_pmi_rule2

,sentence,Noun1,Verb,Noun2
0,chapter introduction project management instit...,[institute],pmi,"[practice, standards]"
1,chapter introduction project management instit...,[knowledge],pmbok,[]
2,chapter introduction project management instit...,[fourth],pmi,[standards]
3,practice standards targeted audiences particip...,[practice],standards,[]
4,practice standards targeted audiences particip...,[management],projects,[]
...,...,...,...,...
1494,.. managing contingency reserves reserves may ...,[reserves],reserves,[]
1495,techniques required allow project manager asse...,[required],level,"[dence, success, project]"
1496,tools managing time buffers closely integrated...,[integrated],projects,[]
1497,tools also required tracking progress spending...,[tracking],progress,[]


## Rule 4

In [209]:
def rule4(text):
    
    doc = nlp(text)
    
    sent = []
    
    for token in doc:
        
        # If the token is a verb
        if (token.pos_=='AUX'):
            
            phrase =''
            
            # Only extract noun or pronoun subjects
            for sub_tok in token.lefts:
                
                if  (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    
                    # Add subject to the phrase
                    phrase += sub_tok.text

                    # Save the root of the word in phrase
                    phrase += ' '+token.lemma_ 

                    # Check for noun or pronoun direct objects
                    for sub_tok in token.rights:
                        
                        # Save the object in the phrase
                        if (sub_tok.pos_ in ['NOUN','PROPN']):
                                    
                            phrase += ' '+sub_tok.text
                            sent.append(phrase)
            
    return sent

In [219]:
output_bok_rule4, output_pmi_rule4 = [],[]

for i in sent_bok:
    output_bok_rule4.append(rule4(text))
    

for i in sent_pmi:
    output_pmi_rule4.append(rule4(i))
    
df_rule4_bok = pd.DataFrame({'sentences':sent_bok,'output':output_bok_rule4})
df_rule4_pmi = pd.DataFrame({'sentences':sent_pmi,'output':output_pmi_rule4})

In [217]:
text='Those actions the project management team can take unilaterally, if the performing organization has a prescribed procurement, contracting, or purchasing department'

### Separate subject, verb and object for PMBOK

In [220]:
verb_dict = dict()
dis_dict = dict()
dis_list = []

# iterating over all the sentences
for i in range(len(df_rule4_bok)):
    
    # sentence containing the output
    sentence = df_rule4_bok.loc[i,'sentences']
    # output of the sentence
    output = df_rule4_bok.loc[i,'output']
    
    # iterating over all the outputs from the sentence
    for sent in output:
        
        # separate subject, verb and object
        n1 = sent.split()[:1]
        v = sent.split()[1]
        n2 = sent.split()[2:]
        
        # append to list, along with the sentence
        dis_dict = {'sentence':sentence,'Noun1':n1,'IS/Have':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep_bok_rule4 = pd.DataFrame(dis_list)

In [221]:
df_sep_bok_rule4

,sentence,Noun1,IS/Have,Noun2
0,project risk management project risk manageme...,[organization],have,[procurement]
1,objectives project risk management increase li...,[organization],have,[procurement]
2,figure provides overview project risk manageme...,[organization],have,[procurement]
3,plan risk managementthe process defining condu...,[organization],have,[procurement]
4,.,[organization],have,[procurement]
...,...,...,...,...
680,"latter responses initially planned , required ...",[organization],have,[procurement]
681,recommended preventive actions .,[organization],have,[procurement]
682,activities ensure future performance project w...,[organization],have,[procurement]
683,... project management plan updates approved c...,[organization],have,[procurement]


### Separate subject, verb and object for PMI

In [201]:
verb_dict = dict()
dis_dict = dict()
dis_list = []

# iterating over all the sentences
for i in range(len(df_rule4_pmi)):
    
    # sentence containing the output
    sentence = df_rule4_pmi.loc[i,'sentences']
    # output of the sentence
    output = df_rule4_pmi.loc[i,'output']
    
    # iterating over all the outputs from the sentence
    for sent in output:
        
        # separate subject, verb and object
        n1 = sent.split()[:1]
        v = sent.split()[1]
        n2 = sent.split()[2:]
        
        # append to list, along with the sentence
        dis_dict = {'sentence':sentence,'Noun1':n1,'IS/Have':v,'Noun2':n2}
        dis_list.append(dis_dict)
        
        # counting the number of sentences containing the verb
        verb = sent.split()[1]
        if verb in verb_dict:
            verb_dict[verb]+=1
        else:
            verb_dict[verb]=1

df_sep_pmi_rule4 = pd.DataFrame(dis_list)

In [203]:
df_sep_pmi_rule4

""


In [ ]:
from bertopic import BERTopic

In [ ]:
model_rse = BERTopic(language="english")

In [ ]:
topics_rse,probs = model_rse.fit_transform(text_bok)

In [ ]:
for i in range(len(topics_rse)):
    print(str(model_rse.get_topic(topics_rse[i]))+"\n")

In [ ]:
badwords=[]
def remove_weak(lists,topics,models):
    for i in range(len(lists)):
        for j in range(len(models.get_topic(topics[i]))):
            if models.get_topic(topics[i])[j][1]==0:
                badwords.append(models.get_topic(topics[i])[j])
            
            
    for test in range(len(topics)):
    
        for i in badwords:
            if i in models.get_topic(topics[test]):
                models.get_topic(topics[test]).remove(i)
remove_weak(text_bok,topics_rse,model_rse)

In [ ]:
import ktrain

In [ ]:
tm_lemma = ktrain.text.get_topic_model(text_bok_lda,n_features=100)

In [ ]:
tm_lemma.build(text_bok_lda,threshold=0.25)

In [ ]:
tm_lemma.print_topics(show_counts=True,)   

In [ ]:
tm_lemma.visualize_documents(doc_topics=tm_lemma.get_doctopics())

In [1]:
from ipynb.fs.full.Rules import rule

In [2]:
df = rule('rule1')

In [3]:
df

,sentence,Noun1,Verb,Noun2
0,project risk management project risk manageme...,[management],includes,[processes]
1,figure provides overview project risk manageme...,[figure],provides,[processes]
2,identify risksthe process determining risks ma...,[process],affect,[project]
3,plan risk responsesthe process developing opti...,[plan],risk,[process]
4,plan risk responsesthe process developing opti...,[opportunities],reduce,[objectives]
...,...,...,...,...
357,frequent discussions risk make likely people i...,[people],identify,[opportunities]
358,... change requests implementing contingency p...,[requests],results,[request]
359,activities ensure future performance project w...,[activities],ensure,[plan]
360,... project management plan updates approved c...,[requests],effect,[processes]


In [62]:
joblib.dump(sent_bok,'bok')
joblib.dump(sent_pmi,'pmi')

['pmi']